# CGMA and GradCAM Generation Notebook
This notebook generates CGMA (Cumulative GradCAM Maps) and GradCAMs for your dataset using a trained classifier.

In [ ]:
# Install required packages
!pip install tensorflow opencv-python matplotlib scikit-learn

In [ ]:
import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

## Define GradCAM and CGMA Functions

In [ ]:
def GradCam(model, img_array, layer_name, eps=1e-8):
    gradModel = tf.keras.models.Model(inputs=[model.inputs], outputs=[model.get_layer(layer_name).output, model.output])
    with tf.GradientTape() as tape:
        inputs = tf.cast(img_array, tf.float32)
        (convOutputs, predictions) = gradModel(inputs)
        loss = predictions[:, 0]
    grads = tape.gradient(loss, convOutputs)
    castConvOutputs = tf.cast(convOutputs > 0, 'float32')
    castGrads = tf.cast(grads > 0, 'float32')
    guidedGrads = castConvOutputs * castGrads * grads
    convOutputs = convOutputs[0]
    guidedGrads = guidedGrads[0]
    weights = tf.reduce_mean(guidedGrads, axis=(0, 1))
    cam = tf.reduce_sum(tf.multiply(weights, convOutputs), axis=-1)
    (w, h) = (img_array.shape[2], img_array.shape[1])
    heatmap = cv2.resize(cam.numpy(), (w, h))
    numer = heatmap - np.min(heatmap)
    denom = (heatmap.max() - heatmap.min()) + eps
    heatmap = numer / denom
    return heatmap

def fuse_layers(layers, model, img):
    cams = []
    for layer in layers:
        cam = GradCam(model, np.expand_dims(img, axis=0), layer)
        cam = cv2.resize(cam, (img.shape[1], img.shape[0]))
        cams.append(cam)
    fused = np.mean(cams, axis=0)
    return fused, cams

def gradcam_ll(layers, model, img):
    layer = layers[-1]
    cam = GradCam(model, np.expand_dims(img, axis=0), layer)
    cam = cv2.resize(cam, (img.shape[1], img.shape[0]))
    return cam

## Set Paths and Load Data

In [ ]:
MODEL_PATH = '../models/Food/VGG/Epoch=04- Loss=0.18 - val_acc = 0.96.h5'
DATASET_PATH = '../Datasets/Food'
# List all image paths
image_paths = [os.path.join(DATASET_PATH, cls, fname) for cls in os.listdir(DATASET_PATH) for fname in os.listdir(os.path.join(DATASET_PATH, cls)) if fname.lower().endswith(('.jpg', '.png'))]
labels = [cls for cls in os.listdir(DATASET_PATH) for fname in os.listdir(os.path.join(DATASET_PATH, cls)) if fname.lower().endswith(('.jpg', '.png'))]
train_a, test_a, train_y, test_y = train_test_split(np.array(image_paths), labels, test_size=0.1, random_state=42, shuffle=False)
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(np.array(train_y).reshape(-1,1))
train_y = enc.transform(np.array(train_y).reshape(-1,1)).toarray()
test_y = enc.transform(np.array(test_y).reshape(-1,1)).toarray()

## Load Model and Get Conv Layers

In [ ]:
model = tf.keras.models.load_model(MODEL_PATH)
model.layers[-1].activation = tf.keras.activations.linear
conv2D_layers = [layer.name for layer in reversed(model.layers) if len(layer.output_shape) == 4 and isinstance(layer, tf.keras.layers.Conv2D)]

## Generate and Save CGMA and GradCAMs

In [ ]:
os.makedirs('../CGMS/Food/VGG', exist_ok=True)
os.makedirs('../Gradcams/Food/VGG', exist_ok=True)
o = len(train_a)
for i in range(o):
    os.makedirs(f'../CGMS/Food/VGG/{i}', exist_ok=True)
    img = cv2.imread(train_a[i])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (256, 256))
    sp, cams = fuse_layers(conv2D_layers, model, img)
    for j, cam in enumerate(cams):
        plt.imsave(f'../CGMS/Food/VGG/{i}/{j}.jpg', cam, cmap='jet')
    print(f'CGMA {i+1}/{o}', end='
')
for i in range(o):
    img = cv2.imread(train_a[i])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (256, 256))
    cam = gradcam_ll(conv2D_layers, model, img)
    plt.imsave(f'../Gradcams/Food/VGG/{i}.jpg', cam, cmap='jet')
    print(f'GradCAM {i+1}/{o}', end='
')